In [ ]:
import numpy as np
from scipy.stats import norm, lognorm, multivariate_normal
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sys
sys.path.append("..")
from src.dataset import CRKPTransmissionSimulator
from src.approx_bc import abc_rejection_sampler
from src.utils import get_results
from timeit import default_timer as timer
import matplotlib.gridspec as gs

In [ ]:
## load data
epi_dir = '/Volumes/umms-esnitkin/Project_KPC_LTACH/Analysis/LTACH_transmission_modeling/preprocessed/resampled'

SCALE = np.array([129., 28., 38., 35., 27., 17., 2])

labels_full = ["Facility", "Floor 1", "Floor 2", "Floor 3", "Floor 4", "SCU", "Room"]

In [ ]:
with open(f"{epi_dir}/observed_data.npy", "rb") as f:
            x_o = (np.load(f).T / SCALE).T

In [ ]:
load_abc_homog = True
load_abc_het = True

# Data Visualization

In [ ]:
crkp_model = CRKPTransmissionSimulator(epi_dir, 0, 1, heterogeneous=False)
x_baseline = crkp_model.CRKP_simulator(-1e8, 100, summarize=True)

infections = pd.read_csv(f"{epi_dir}/infections.csv", index_col=0)
infections.columns=np.arange(53)

In [ ]:
# figure out who is initially infected
T = infections.shape[1]
initial = np.zeros(T)
starting_index = infections.index[infections[0] == 1]
discharged = np.zeros(len(starting_index)).astype(bool)
for t in range(T):
    for i, ix in enumerate(starting_index):
        if discharged[i]:
            continue
        elif np.isnan(infections.loc[ix, t]):
            # print(ix, t)
            discharged[i] = True
    initial[t] = (1 - discharged).sum()

In [ ]:
NN = infections.notna().sum(0)
II = (infections == 1).sum(0)
sns.lineplot(NN, linestyle="--", label="Total")
sns.lineplot(II, label="Infected")
sns.lineplot(x_baseline * SCALE[0] - initial, linestyle=":", label="Screened Cases")
sns.lineplot(initial, linestyle="dashdot", label="Initial Cases")
plt.ylim(bottom=0)
plt.ylabel("Patients")
plt.xlabel("Weeks")
# plt.savefig("images/crkp/crkp.png")
plt.xlim(right=53)
plt.show()

In [ ]:
floor_pops = []
for i in [1,2,3,4,5]:
    floor_pops.append((crkp_model.F == i).sum(0))
floor_pops = np.stack(floor_pops)

floor_cases = x_o[1:6,]

In [ ]:
floor_pops.mean(1)

In [ ]:
labels=["Floor 1", "Floor 2", "Floor 3", "Floor 4", "SCU"]
for i, label  in enumerate(labels):
    # temp_pop = np.where(floor_pops[i] == 0, 1, floor_pops[i])
    plt.plot(floor_pops[i], label=label, linestyle="-", alpha=0.6)
plt.legend()
plt.xlabel("Weeks")
plt.ylabel("Patients")
# plt.savefig("images/crkp/floor_pops.png")
plt.show()

In [ ]:
labels=["Floor 1", "Floor 2", "Floor 3", "Floor 4", "SCU"]
for i, label  in enumerate(labels):
    # temp_pop = np.where(floor_pops[i] == 0, 1, floor_pops[i])
    plt.plot(floor_cases[i], label=label, linestyle="--")
plt.legend()
plt.show()

In [ ]:
# average case burden
for i, lab in enumerate(labels):
    temp_pop = np.where(floor_pops[i] == 0, 1, floor_pops[i])
    avg_burden = np.median(floor_cases[i] * SCALE[1:6][i] / temp_pop) # median is clearer
    print(f"{lab}: {avg_burden:.2f}")

print(np.median(II / NN))

# NPE

## Homogeneous

In [ ]:
prior_mu = -2
prior_sigma = 1
crkp_model = CRKPTransmissionSimulator(epi_dir, prior_mu, prior_sigma, heterogeneous=False)

In [ ]:
mu = -2.056
sigma = 0.124

d = norm(loc = mu, scale = sigma)
print(np.exp(d.ppf(0.05)), np.exp(d.ppf(0.95)))
print(np.exp(mu + sigma**2/2))

In [ ]:
np.exp(-2.056 + 0.124**2/2)

In [ ]:
prior_mean = np.exp(prior_mu + prior_sigma**2/2)
prior = norm(prior_mu, prior_sigma)
x = np.arange(0., .5, 0.001)
ax1 = sns.lineplot(x=x, y=lognorm.pdf(x, scale=np.exp(prior_mu), s=prior_sigma), 
                   label="Prior Density", linestyle=":")
posterior_mean = np.exp(mu + sigma**2/2)
plt.axvline(prior_mean, color="k", linestyle=":", label="Prior Mean")
ax = sns.lineplot(x=x, y=lognorm.pdf(x, scale=np.exp(mu), s=sigma), label="Neural Posterior Density")
plt.axvline(posterior_mean, color="k", label="Neural Posterior Mean")
ax.set_xlabel(r"Infection Rate $\beta$")
ax.set_yticks([])
plt.legend()
# plt.savefig("images/crkp_homog.png")
plt.show()

### prior predictive checks

In [ ]:
prior = norm(prior_mu, prior_sigma)
K = 30
T = x_o.shape[1]
prior_sample = prior.rvs(size=K, random_state=2)
prior_predictive = np.empty((K, 7, T))
for i in range(K):
    x_rep = crkp_model.CRKP_simulator(prior_sample[i], i, True) * np.repeat(SCALE[:, None], T, 1)
    prior_predictive[i] =  np.array(x_rep)

In [ ]:
for j in range(7):
    pp_mean = prior_predictive.mean(0)[j]
    labels = ["Prior Pred. Draw"] + [None for _ in range(K-1)]
    I_o = np.array(x_o)[j] * SCALE[j]
    plt.plot(I_o, label="Observed", color="k")
    plt.plot(prior_predictive[:, j, :].T, 
             label=labels, color="b",
             alpha=0.1)
    plt.plot(pp_mean, label="Prior. Pred. Mean", linestyle="--",
             color="orange")
    plt.legend()
    if j == 6:
        ylab = "Infected Roommates"
    else:
        ylab = "Infected Patients"
    plt.ylabel(ylab)
    plt.xlabel("Time Steps")
    # plt.savefig("images/crkp_ppc1.png")
    plt.show()

### posterior predictive checks

In [ ]:
neural_posterior = norm(mu, sigma)
npe_sample = neural_posterior.rvs(size=K, random_state=2)
posterior_predictive = np.empty((K,7,T))
for i in range(K):
    x_rep = crkp_model.CRKP_simulator(npe_sample[i], i, True) * np.repeat(SCALE[:, None], T, 1)
    posterior_predictive[i] =  np.array(x_rep)

In [ ]:
for j in range(7):
    pp_mean = posterior_predictive.mean(0)[j]
    labels = ["Post. Pred. Draw"] + [None for _ in range(K-1)]
    I_o = np.array(x_o)[j] * SCALE[j]
    plt.plot(I_o, label="Observed", color="k")
    plt.plot(posterior_predictive[:, j, :].T, 
             label=labels, color="b",
             alpha=0.1)
    plt.plot(pp_mean, label="Post. Pred. Mean", linestyle="--",
             color="orange")
    plt.legend()
    if j == 6:
        ylab = "Infected Rooms"
    else:
        ylab = "Infected Patients"
    plt.ylabel(ylab)
    plt.xlabel("Weeks")
    plt.savefig(f"images/crkp/crkp_ppc_homog{j}.png")
    # plt.ylim(0, 135)
    plt.show()

## Heterogeneous

In [ ]:
prior_mu = -3
prior_sigma = 1

#### prior predictive checks

In [ ]:
crkp_model = CRKPTransmissionSimulator(epi_dir, prior_mu, prior_sigma, heterogeneous=True)
prior_het = multivariate_normal(
    np.full((7,), prior_mu), 
    np.diag(np.full((7,), prior_sigma))
)
K = 30
T = x_o.shape[1]
prior_sample_het = prior_het.rvs(size=K, random_state=2)
prior_predictive_het = np.empty((K, 7, T))
for i in range(K):
    x_rep = crkp_model.CRKP_simulator(prior_sample_het[i], i) * np.repeat(SCALE[:, None], T, 1)
    prior_predictive_het[i] = np.array(x_rep)

In [ ]:
for j in range(7):
    pp_mean = prior_predictive_het.mean(0)[j]
    labels = ["Prior Pred. Draw"] + [None for _ in range(K-1)]
    I_o = np.array(x_o)[j] * SCALE[j]
    plt.plot(I_o, label="Observed", color="k")
    plt.plot(prior_predictive_het[:, j, :].T, 
             label=labels, color="b",
             alpha=0.1)
    plt.plot(pp_mean, label="Prior Pred. Mean", linestyle="--",
             color="orange")
    plt.legend()
    if j == 6:
        ylab = "Infected Roommates"
    else:
        ylab = "Infected Patients"
    plt.ylabel(ylab)
    plt.xlabel("Weeks")
    if j == 0:
        pass
        # plt.savefig(f"images/crkp/crkp_prior_pc.png")
    plt.show()

In [ ]:
# df_het = get_results("../multirun/2024-12-04/14-09-03") 
# df_het.sort_values("val_loss", inplace=True)

df_het = get_results("../multirun/2025-03-05/17-58-13/")
df_het.sort_values("val_loss", inplace=True)

In [ ]:
mu = df_het["mu"][0][0]
sigma = df_het["sigma"][0]

In [ ]:
m1 = np.array(mu)
s1 = np.array([sigma[i][i] for i in range(7)])
rates = np.exp(mu + s1/2)

In [ ]:
# rough R_0 estimate
np.sum(SCALE[1:5] * rates[1:5]) / np.sum(SCALE[1:5]) + rates[0] + rates[-1]

#### posterior predictive checks

In [ ]:
crkp_model = CRKPTransmissionSimulator(epi_dir, prior_mu, prior_sigma, heterogeneous=True)
neural_posterior_het = multivariate_normal(mu, sigma)
npe_sample_het = neural_posterior_het.rvs(size=K, random_state=2)
posterior_predictive_het = np.empty((K, 7, T))
for i in range(K):
    x_rep = crkp_model.CRKP_simulator(npe_sample_het[i], i) * np.repeat(SCALE[:, None], T, 1) 
    posterior_predictive_het[i] =  np.array(x_rep)

In [ ]:
for j in range(7):
    pp_mean = posterior_predictive_het.mean(0)[j]
    labels = ["Post. Pred. Draw"] + [None for _ in range(K-1)]
    I_o = np.array(x_o)[j] * SCALE[j]
    plt.plot(I_o, label="Observed", color="k")
    plt.plot(posterior_predictive_het[:, j, :].T, 
             label=labels, color="b",
             alpha=0.1)
    plt.plot(pp_mean, label="Post. Pred. Mean", linestyle="--",
             color="orange")
    # sns.lineplot(NN, color="green", linestyle="--", label="Total")
    plt.legend()
    if j == 6:
        ylab = "Infected Rooms"
    else:
        ylab = "Infected Patients"
    plt.ylabel(ylab)
    plt.xlabel("Weeks")
    # plt.savefig(f"images/crkp/crkp_ppc_het{j}.png")
    plt.show()

In [ ]:
for j in range(7):
    pp_mean = posterior_predictive_het.mean(0)[j]
    labels = ["Post. Pred. Draw"] + [None for _ in range(K-1)]
    I_o = np.array(x_o)[j] * SCALE[j]
    plt.plot(I_o, label="Observed", color="k")
    plt.plot(posterior_predictive_het[:, j, :].T, 
             label=labels, color="b",
             alpha=0.1)
    plt.plot(pp_mean, label="Post. Pred. Mean", linestyle="--",
             color="orange")
    # sns.lineplot(NN, color="green", linestyle="--", label="Total")
    plt.legend()
    if j == 6:
        ylab = "Infected Rooms"
    else:
        ylab = "Infected Patients"
    plt.ylabel(ylab)
    plt.xlabel("Weeks")
    # plt.savefig(f"images/crkp/crkp_ppc_het{j}.png")
    plt.show()

## Interventions

In [ ]:
pis = [
    [.2, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, .2, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, .2],
    [.2, .2, .2, .2, .2, .2, 1],
    [1e-8, 1e-8, 1e-8, 1e-8, 1e-8, 1e-8, 1e-8]
]
intervention = np.empty((K, len(pis), T))
case_counts = np.empty((K, len(pis)))
for j, pi in enumerate(pis):
    crkp_model = CRKPTransmissionSimulator(epi_dir, prior_mu, prior_sigma, heterogeneous=True, pi=pi, return_case_count=True)
    for i in range(K):
        x_rep, cc = crkp_model.CRKP_simulator(npe_sample_het[i], i * 2)
        intervention[i,j] =  np.array(x_rep)[0] * SCALE[0]
        case_counts[i, j] = cc
intervention = intervention.mean(0)
case_counts = case_counts.mean(0)

In [ ]:
case_counts

In [ ]:
intervention.sum(1)

In [ ]:
# plt.plot(np.array(x_o)[0] * SCALE[0], label="Observed", color="k")
plt.plot(posterior_predictive_het.mean(0)[0], label="No Intervention", linestyle="-")
plt.plot(intervention[0], label="Floor Isolation", linestyle="--")
plt.plot(intervention[1], label="Target Floor 3", linestyle="--")
plt.plot(intervention[3], label="Room Isolation", linestyle="--")
plt.plot(intervention[4], label="Zero Transmission", linestyle=":")
# plt.plot(intervention[3], label="Roomies", linestyle="--")
plt.legend()
plt.xlabel("Weeks")
plt.ylabel("Number Infected")
plt.savefig("images/crkp/crkp_intervention.png")
plt.show()

In [ ]:
df_cc.index

In [ ]:
df_cc.reset_index()

In [ ]:
df_cc = pd.DataFrame(
    data = {"Scenario": ["No Intervention", "Floor Isolation", "Target Floor 3", "Room Isolation", "Zero Transmission"],
        "Total Infections": [259, 230, 229, 199, 168]}
)
sns.barplot(df_cc, x = "Total Infections", y = "Scenario", hue="Scenario", legend=False, alpha=0.8,
           width=0.62).set(ylabel="")
plt.savefig("images/crkp/crkp_case_count.png", bbox_inches='tight')
plt.plot()

In [ ]:
zzz = np.array([259, 230, 229, 199, 168])
df_cc_alt = pd.DataFrame(
    data = {"Scenario": ["No Intervention", "Floor Isolation", "Target Floor 3", "Room Isolation", "Zero Transmission"],
        "Total Secondary Infections": zzz - 168}
)
sns.barplot(df_cc_alt.head(4), x = "Total Secondary Infections", y = "Scenario", hue="Scenario", legend=False, alpha=0.8,
           width=0.62).set(ylabel="")
plt.savefig("images/crkp/crkp_case_count2.png", bbox_inches='tight')
plt.plot()

In [ ]:
168 + 91

In [ ]:
\begin{table}[]
\begin{tabular}{@{}ll@{}}
\toprule
Scenario          & Total Infections \\ \midrule
Factual           & 259              \\
Floor Isolation   & 230              \\
Target Floor 3    & 229              \\
Room Isolation    & 199              \\
Zero Transmission & 168              \\ \bottomrule
\end{tabular}
\end{table}

In [ ]:
grid = gs.GridSpec(8, 1)
fig = plt.figure(figsize=(5,7))
x_min = -6
x_max = -0
x = np.arange(x_min, x_max, 0.05)
axes = []
alpha = 0.5
for i in range(7):
    legend = False if i > 0 else True
    axes.append(fig.add_subplot(grid[i:i+1, 0:]))

    priory = norm.pdf(x, prior_mu, np.sqrt(prior_sigma))
    posty = norm.pdf(x, mu[i], np.sqrt(sigma[i][i]))
    sns.lineplot(x=x, y=priory, label="Prior Density", ax=axes[-1], legend=legend, alpha=alpha)

    sns.lineplot(x=x, y=posty, label="Posterior Density", ax=axes[-1], legend=legend, alpha=alpha)

    rect = axes[-1].patch
    rect.set_alpha(0)

    axes[-1].set_yticklabels([])
    axes[-1].set_ylabel('')

    axes[-1].set_ylim(0, 2)
    axes[-1].set_xlim(x_min, x_max)

    axes[-1].set_yticks([])

    if i < 7 - 1:
        axes[-1].set_xticklabels([])
    else:
        axes[-1].set_xlabel("Infection Rate (Log-scale)")

    axes[-1].fill_between(x, posty, color='#ff7f0e', alpha=alpha)
    axes[-1].fill_between(x, priory, color='#1f77b4', alpha=alpha)


    axes[-1].text(x_min - .8, 0.3, labels_full[i])
grid.update(hspace= -.0)
plt.tight_layout()
plt.savefig("images/crkp/crkp_compare_het.png")
plt.show()
    

In [ ]:
D = np.diag(1 / np.sqrt(np.diag(sigma)))
corr = D @ sigma @ D
plt.matshow(corr, cmap="rocket")
plt.colorbar()
plt.xlabel(r"$\beta_j$")
plt.ylabel(r"$\beta_j$")
plt.savefig("images/crkp/crkp_corr.png")
plt.show()

# ABC Comparison

## Homogeneous

In [ ]:
hetero = False

prior_mu = -2
prior_sigma = 1
model = CRKPTransmissionSimulator(epi_dir, prior_mu, prior_sigma,
                                  heterogeneous=hetero)

In [ ]:
prior_sampler = lambda: model.sample_logbeta(1)
simulator = lambda theta, seed: model.CRKP_simulator(theta, seed, summarize=True)

S = 100
epsilon = .2
posterior_sample, errors = abc_rejection_sampler(
    S, epsilon, prior_sampler, simulator, x_o[0], max_attempts=10000,
    summarize=False, print_every=1000
)

In [ ]:
np.mean(posterior_sample), np.std(posterior_sample)

In [ ]:
np.exp(posterior_sample).mean()

## Heterogeneous

In [ ]:
summarize = False #True
hetero = True

prior_mu = -3
prior_sigma = 1
model = CRKPTransmissionSimulator(epi_dir, prior_mu, prior_sigma,
                                  heterogeneous=hetero)

In [ ]:
if load_abc_het:
    posterior_sample_het = np.load("crkp_abc_het.npy")
else:
    prior_sampler = lambda: model.sample_logbeta(1)
    simulator = lambda theta, seed: model.CRKP_simulator(theta, seed, summarize=True)
    
    S = 100
    epsilon = 9
    posterior_sample_het, errors = abc_rejection_sampler(
        S, epsilon, prior_sampler, simulator, x_o, max_attempts=20000,
        summarize=summarize, print_every=1000
    )
    np.save("crkp_abc_het", posterior_sample_het)

Latest run: $\epsilon = 9$ , number of samples 13,214

In [ ]:
# sns.pairplot(pd.DataFrame(posterior_sample))

In [ ]:
abc_corr = np.corrcoef(posterior_sample_het.T)
plt.matshow(abc_corr, cmap="rocket")
plt.colorbar()
plt.xlabel(r"$\beta_j$")
plt.ylabel(r"$\beta_j$")
# plt.savefig("images/crkp/abc_corr.png")
plt.show()

### Posterior Predictive Checks

In [ ]:
K = 30
crkp_model = CRKPTransmissionSimulator(epi_dir, prior_mu, prior_sigma, heterogeneous=True)
# neural_posterior_het = multivariate_normal(mu, sigma)
# npe_sample_het = neural_posterior_het.rvs(size=K, random_state=2)
posterior_predictive_abc_het = np.empty((K, 7, T))
for i in range(K):
    x_rep = crkp_model.CRKP_simulator(posterior_sample[i], i, summarize=True) * np.repeat(SCALE[:, None], T, 1) 
    posterior_predictive_abc_het[i] =  np.array(x_rep)

In [ ]:
for j in range(7):
    preds = posterior_predictive_abc_het[:, j, :]
    target = (x_o[j] * SCALE[j])
    mse = ((preds - target)**2).mean()
    print(round(mse, 3))

In [ ]:
het_table = {"Rate": labels_full, 
             "NPE Mean": np.array(rates), 
             "NPE CI": [],
            "ABC Mean": np.exp(posterior_sample).mean(0),
            "ABC CI": []}

for i in range(7):
    dist = norm(mu[i], np.sqrt(sigma[i][i]))
    het_table["NPE CI"].append(
        (float(np.exp(dist.ppf(0.05))), 
         float(np.exp(dist.ppf(0.95))))
    )
    het_table["ABC CI"].append(
        (float(np.quantile(np.exp(posterior_sample), 0.05, axis=0)[i]),
        float(np.quantile(np.exp(posterior_sample), 0.95, axis=0)[i]))
    )
het_table = pd.DataFrame(het_table)
print(het_table.to_latex(index=False))

In [ ]:
np.exp(posterior_sample).mean(0)

In [ ]:
crkp_model = CRKPTransmissionSimulator(epi_dir, prior_mu, prior_sigma, heterogeneous=True)
# neural_posterior_het = multivariate_normal(mu, sigma)
# npe_sample_het = neural_posterior_het.rvs(size=K, random_state=2)
posterior_predictive_abc = np.empty((K, 7, T))
for i in range(K):
    x_rep = crkp_model.CRKP_simulator(posterior_sample[i], i) * np.repeat(SCALE[:, None], T, 1) 
    posterior_predictive_abc[i] =  np.array(x_rep)

In [ ]:
for j in range(7):
    preds = posterior_predictive_abc[:, j, :]
    target = (x_o[j] * cap[j])
    mse = ((preds - target)**2).mean()
    print(round(mse, 3))

In [ ]:
for j in range(7):
    pp_mean = posterior_predictive_abc.mean(0)[j]
    labels = ["Post. Pred. Draw"] + [None for _ in range(K-1)]
    I_o = np.array(x_o)[j] * SCALE[j]
    plt.plot(I_o, label="Observed", color="k")
    plt.plot(posterior_predictive_abc[:, j, :].T, 
             label=labels, color="b",
             alpha=0.1)
    plt.plot(pp_mean, label="Post. Pred. Mean", linestyle="--",
             color="orange")
    # sns.lineplot(NN, color="green", linestyle="--", label="Total")
    plt.legend()
    if j == 6:
        ylab = "Infected Rooms"
    else:
        ylab = "Infected Patients"
    plt.ylabel(ylab)
    plt.xlabel("Weeks")
    # plt.savefig(f"images/crkp/crkp_ppc_abc{j}.png")
    plt.show()

In [ ]:
grid = gs.GridSpec(8, 1)
fig = plt.figure(figsize=(5,7))
xmin, xmax = -6, 0

x = np.arange(xmin, xmax, 0.05)
axes = []
alpha = 0.5
for i in range(7):
    legend = False if i > 0 else True
    axes.append(fig.add_subplot(grid[i:i+1, 0:]))

    priory = norm.pdf(x, prior_mu, np.sqrt(prior_sigma))
    # posty = norm.pdf(x, mu[i], np.sqrt(sigma[i][i]))
    sns.lineplot(x=x, y=priory, label="Prior Density", ax=axes[-1], legend=legend, alpha=alpha)

    sns.histplot(posterior_sample[:, i], color="orange", ax=axes[-1], legend=legend, alpha=alpha, stat="density",
                kde=True, label="ABC Posterior")

    post_mean = np.mean(posterior_sample[:, i])

    # plt.axvline(post_mean, color="k", linestyle="--")

    # sns.lineplot(x=x, y=posty, label="Posterior Density", ax=axes[-1], legend=legend, alpha=alpha)

    rect = axes[-1].patch
    rect.set_alpha(0)

    axes[-1].set_yticklabels([])
    axes[-1].set_ylabel('')

    axes[-1].set_ylim(0, 2)
    axes[-1].set_xlim(xmin, xmax)

    axes[-1].set_yticks([])

    if i < 7 - 1:
        axes[-1].set_xticklabels([])
    else:
        axes[-1].set_xlabel("Infection Rate (Log-scale)")

    # axes[-1].fill_between(x, posty, color='#ff7f0e', alpha=alpha)
    axes[-1].fill_between(x, priory, color='#1f77b4', alpha=alpha)


    axes[-1].text(x_min - 0.8, 0.3, labels_full[i])

    if i == 0:
        plt.legend()
grid.update(hspace= -.0)
plt.tight_layout()
plt.savefig("images/crkp/crkp_compare_abc.png")
plt.show()

# relative risks

In [ ]:
rates

In [ ]:
rates / SCALE

In [ ]:
SCALE

In [ ]:
(rates / SCALE) / (rates / SCALE)[0]

In [ ]:
abc_rates = np.exp(posterior_sample).mean(0)

In [ ]:
(abc_rates / SCALE) / (abc_rates / SCALE)[0]

In [ ]:
(np.exp(-2.5) / SCALE) / (np.exp(-2.5) / SCALE[0])

In [ ]:
np.log([.0352, .322, .0153, .191])

In [ ]:
np.log([.0094, .251, .0109, .154])

In [ ]:
np.log([.00820, .302, .0128, .242])

In [ ]:
-1.19 + 4.8, -1.4 + 3.4

In [ ]:
- 1.1 + 3.3

In [ ]:
-1.65 + 4.2